This notebook will be mainly used for the capstone project.

In [18]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



In [26]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium # map rendering library
import requests # library to handle requests
import random # library for random number generation
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas import json_normalize
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
CLIENT_ID = 'CPKUF5JGHYZMWBIX1BMCK3QM30NIHWJ2IDCQHN1CRBX4UW2G' # your Foursquare ID
CLIENT_SECRET = 'A5M21YUKPT00LNTFW42W0SNORNWXQEBGCH0EK31SRGEEIALF' # your Foursquare Secret
ACCESS_TOKEN = '1S4LQ2H4DTD411ZHPV3QWDMNPGKNPX2OWH4AL1QZJZHCYSSF' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CPKUF5JGHYZMWBIX1BMCK3QM30NIHWJ2IDCQHN1CRBX4UW2G
CLIENT_SECRET:A5M21YUKPT00LNTFW42W0SNORNWXQEBGCH0EK31SRGEEIALF


In [3]:
cat_dict = {"Country Dance Club": "52e81612bcbc57f1066b79ef", 
            "Dance Studio":"4bf58dd8d48988d134941735", 
            "Salsa Club": "52e81612bcbc57f1066b79ec", 
            "Gym":"4bf58dd8d48988d176941735", 
            "Pilates Studio":"5744ccdfe4b0c0459246b4b2", 
            "Yoga Studio": "4bf58dd8d48988d102941735", 
             "Skating Rink":"4bf58dd8d48988d168941735", 
            "Lake": "4bf58dd8d48988d161941735", 
            "Concert Hall": "5032792091d4c4b30a586d5c",
           "Hot Spring": "4bf58dd8d48988d160941735"}

In [4]:
def venues(categories, city):
    ids = ",".join([cat_dict[x] for x in categories])
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&oauth_token={}&v={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, city, ACCESS_TOKEN, VERSION, LIMIT, ids)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df = pd.json_normalize(venues)

    def clean_df(df):
        filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
        new_df = df.loc[:, filtered_columns]

        def get_category_type(row):
            try:
                categories_list = row['categories']
            except:
                categories_list = row['venue.categories']

            if len(categories_list) == 0:
                return None
            else:
                return categories_list[0]['name']

        new_df['categories'] = new_df.apply(get_category_type, axis=1)
        new_df.columns = [column.split('.')[-1] for column in new_df.columns]
        new_df = new_df[new_df['categories'].isin(categories)]
        return new_df

    return clean_df(df)

In [5]:
cities = ["Billings, MT", "Missoula, MT", "Great Falls, MT", "Bozeman, MT", "Boise, ID", "Meridian, ID", "Nampa, ID", "Coeur d'Alene, ID", "Fargo, ND", "Sioux Falls, SD", "Rapid City, SD", "Lincoln, NE", "Omaha, NE", "Bellevue, NE", "Des Moines, IA", "Cedar Rapids, IA", "Cheyenne, WY", "Casper, WY", "Laramie, WY"]
categories = ["Gym", "Yoga Studio", "Pilates Studio", "Country Dance Club", "Dance Studio", "Salsa Club", "Skating Rink", "Lake", "Concert Hall", "Hot Spring"]

In [6]:
bozeman_df = venues(categories, "Bozeman, MT")

In [7]:
bozeman_df.sort_values("categories")

,name,categories,address,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
31,Eagles Ballroom,Concert Hall,NaN,45.679017,-111.032449,"[{'label': 'display', 'lat': 45.679017, 'lng':...",59715,US,Bozeman,MT,United States,"[Bozeman, MT 59715]",NaN,50c40968e4b065e8ec4070a8
40,Main Street Dance Theater,Dance Studio,NaN,45.703150,-111.045088,"[{'label': 'display', 'lat': 45.70315, 'lng': ...",59715,US,Bozeman,MT,United States,"[Bozeman, MT 59715]",NaN,5089ef58e4b08e7b4239f38a
33,Have Fun Dancing,Dance Studio,NaN,45.695731,-111.030763,"[{'label': 'display', 'lat': 45.69573058736429...",59715,US,Bozeman,MT,United States,"[Bozeman, MT 59715]",NaN,50c14d2ee4b0276c8694b8b1
32,Montana Ballet Company,Dance Studio,521 E Peach St,45.685972,-111.029716,"[{'label': 'display', 'lat': 45.685972, 'lng':...",59715,US,Bozeman,MT,United States,"[521 E Peach St, Bozeman, MT 59715]",NaN,4f3d93b7e4b0bfb6302a666c
17,Romney Gym Dance Studio,Dance Studio,NaN,45.665698,-111.049550,"[{'label': 'display', 'lat': 45.665698, 'lng':...",59717,US,Bozeman,MT,United States,"[Bozeman, MT 59717]",NaN,57c650d4498e692f4ac74714
20,Bozeman Dance Academy,Dance Studio,346 Gallatin Park Dr,45.707058,-111.044891,"[{'label': 'display', 'lat': 45.70705795288086...",59715,US,Bozeman,MT,United States,"[346 Gallatin Park Dr, Bozeman, MT 59715]",NaN,5be91846cbcdee002c905141
0,The Ridge Downtown,Gym,Mendenhall,45.680712,-111.035276,"[{'label': 'display', 'lat': 45.680712, 'lng':...",59715,US,Bozeman,MT,United States,"[Mendenhall, Bozeman, MT 59715]",NaN,4c58b8b3b8b3c9b6f29b87b6
39,Ridgecrest,Gym,NaN,45.680714,-111.035218,"[{'label': 'display', 'lat': 45.680714, 'lng':...",59715,US,Bozeman,MT,United States,"[Bozeman, MT 59715]",NaN,505cb15ce4b0bcb2133b70ae
36,Health Balance,Gym,NaN,45.658800,-111.055470,"[{'label': 'display', 'lat': 45.6588, 'lng': -...",NaN,US,Bozeman,MT,United States,"[Bozeman, MT]",NaN,5193dd13498e0397c2193f2a
35,The Rock,Gym,Amsterdam Road,45.774171,-111.219251,"[{'label': 'display', 'lat': 45.77417094730278...",59714,US,Belgrade,MT,United States,"[Amsterdam Road, Belgrade, MT 59714]",NaN,4f09dc96e4b0aa2dc7cea6c5


In [33]:
def city_dfs(cities):
    for city in cities:
        df = venues(categories, city)
        df.to_csv("{}.csv".format(city))

In [8]:
def venue_percentages(categories, cities):
    def perc(dic, key, denom):
        value = dic.get(key)
        if value is None:
            return 0
        else:
            return value / denom
    data = {}
    for city in cities:
        city_df = venues(categories, city)
        venue_count = city_df.groupby('categories').count()['name'].to_dict()
        venue_perc = dict([(category, perc(venue_count, category, city_df.shape[0])) for category in categories])
        data[city] = venue_perc
    df = pd.DataFrame(data).T
    for category in categories:
        if sum(df[category]) == 0:
            del df[category]
    return df

In [9]:
df = venue_percentages(categories, cities)

In [10]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000
"Missoula, MT",0.525000,0.125000,0.000000,0.00000,0.150000,0.025,0.025000,0.050000,0.050000,0.050000
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000
"Bozeman, MT",0.378378,0.189189,0.027027,0.00000,0.135135,0.000,0.027027,0.189189,0.027027,0.027027
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641
"Nampa, ID",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.000000,0.128205,0.076923,0.025641
"Coeur d'Alene, ID",0.219512,0.000000,0.000000,0.00000,0.073171,0.000,0.000000,0.707317,0.000000,0.000000
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000


In [11]:
def add_clusters(df, clusters):
    new_df = df
    k_means = KMeans(init="k-means++", n_clusters=clusters, n_init=12)
    k_means.fit(new_df)
    new_df["Labels"] = k_means.labels_
    return new_df.sort_values("Labels")

In [12]:
df = add_clusters(df, 5)

In [13]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1


In [31]:
address = 'Lincoln, NE'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Lincoln, NE are 41.1280799, -101.1313806.


In [21]:
def add_lats_lngs(df):
    new_df = df
    geolocator = Nominatim(user_agent="explorer")
    lats = {}
    lngs = {}
    for city in cities:
        location = geolocator.geocode(city)
        latitude = location.latitude
        longitude = location.longitude
        lats[city] = latitude
        lngs[city] = longitude
    new_df["Latitude"] = pd.Series(lats)
    new_df["Longitude"] = pd.Series(lngs)
    return new_df

    

In [36]:
df = add_lats_lngs(df)

In [37]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels,Latitude,Longitude
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0,41.975887,-91.670405
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0,41.591032,-93.604665
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0,13.547005,2.112145
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0,41.258746,-95.938376
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1,45.787496,-108.496070
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1,46.877229,-96.789821
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1,43.546059,-96.731393
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1,43.608629,-116.392326
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1,43.616616,-116.200886
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1,47.504885,-111.291890


In [38]:
def cluster_map(df):  
    map_clusters = folium.Map(location=[np.mean(df["Latitude"]), np.mean(df["Longitude"])], zoom_start=11)

    # set color scheme for the clusters
    kclusters = np.max(df["Labels"]+1)
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for city, lat, lon, cluster in zip(df.index.values, df['Latitude'], df['Longitude'], df['Labels']):
        label = folium.Popup(str(city) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

In [39]:
my_map = cluster_map(df)

In [40]:
my_map

In [46]:
df.at["Lincoln, NE", "Latitude"] = 40.813618

In [47]:
df.at["Lincoln, NE", "Longitude"] = -96.702599

In [48]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels,Latitude,Longitude
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0,41.975887,-91.670405
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0,41.591032,-93.604665
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0,13.547005,2.112145
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0,41.258746,-95.938376
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1,45.787496,-108.496070
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1,46.877229,-96.789821
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1,43.546059,-96.731393
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1,43.608629,-116.392326
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1,43.616616,-116.200886
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1,47.504885,-111.291890


In [49]:
my_map = cluster_map(df)

In [50]:
my_map

In [51]:
def relative_percentages_per_group(df):
    df_gb = df.groupby('Labels').mean()
    for category in df_gb.columns:
        total = sum(df_gb[category])
        df_gb[category] = [value / total for value in df_gb[category]]
    return df_gb

In [52]:
df_gb = relative_percentages_per_group(df)

In [53]:
df_gb

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Latitude,Longitude
Labels,,,,,,,,,,,,
0,0.182201,0.074784,0.000000,1.0,0.097480,0.0,0.315628,0.280109,0.091928,0.295556,0.161148,0.136837
1,0.287308,0.203095,0.291268,0.0,0.129392,0.0,0.171900,0.091630,0.401262,0.179374,0.209315,0.213610
2,0.184889,0.330277,0.708732,0.0,0.251770,0.0,0.243284,0.147165,0.355128,0.203311,0.198476,0.204498
3,0.106052,0.000000,0.000000,0.0,0.153717,0.0,0.000000,0.444442,0.000000,0.000000,0.222085,0.229020
4,0.239550,0.391843,0.000000,0.0,0.367640,1.0,0.269188,0.036654,0.151682,0.321760,0.208976,0.216035
